# Implementing RNN-LM "From Scratch"

## Question 1
Explain design principles of input/output representations, word embedding, time step
structure for feeding hidden states, weight matrix management for feedforward and backpropagation, BPTT(Backpropagation Through Time) algorithm, and optimization(SGD, Adam, or AdaGrad)

** Answer: **
The input and output of RNN are vectors. We first build a dictionary and replace the string with index

Word embeddings is to map a string word to a vector of numbers, here I use one hot emmbeding. For example if I use a vocab with size 15000, dictionary["this"] = 3, then a vector [0, 0, 0, 1, ...,0, 0] with dim euqals to 5000 will represent the input word "this"

For RNN we have 
$$s_{cur} = tanh(Ux_{cur} + Ws_{pre})$$
$$o_{cur} = softmax(Vs_{cur})$$
where $s_{cur}$ is the current state

$s_{prev}$ is the prev state

$x_{cur}$ is the current input

$U$, $V$, $W$ are the shared parameter matrixes across the time, action as input gate, output gate and write gate.

In this experiment I choose vocab size = 15000, hidden layer size = 100, so we have:

intput x.shape = $\mathbb{R}^{15000}$

output o.shape = $\mathbb{R}^{15000}$

state s.shape = $\mathbb{R}^{100}$

input weight matrix $U$.shape = $\mathbb{R}^{100\times15000}$

output weight matrix $V$.shape = $\mathbb{R}^{15000\times100}$

write weight matrix $W$.shape = $\mathbb{R}^{100\times100}$

For BPTT, the only different with standard back propagation algorithm is we sum up the gradients for $W$ at each time step, For optimization I use SGD, so in each training iteration I will calculate the gradient of that batch, not the whole dataset to save time. The disadvantage is gradient will may change dramatically from iteration to iteration and hard to decide the learning rate. 

AdaGrad updates the gradient by combine current gradient with the history gradients. It adapts the learning rate to the parameter. The basic idea is if the sum of history gradients is large(means we already learn a lot on that dimension), the current gradient will contribute less to the weight updating.

In [1]:
# Import lib
import zipfile                                                                                                                                                            
import os                                                                                                                                                
import numpy as np                                                                                                                                                        
import random                                                                                                                                                             
import math                                                                                                                                                               
import collections 
import operator
import datetime
import sys
from six.moves.urllib.request import urlretrieve

In [2]:
# Download data
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
# Load data
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = f.read(f.namelist()[0]).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [4]:
##### Build vocab and inverse vocab
vocabulary_size = 15000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Vocab size %d' % len(dictionary))

Vocab size 15000


In [ ]:
# Build training data.
# Each training instance is the word in dataset, label is the next word right after the training word
data_index = 0

def generate_batch(training_size, batch_size):
  global data_index

  training_data = np.ndarray(shape=(training_size, batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(training_size, batch_size), dtype=np.int32)
  buffer = collections.deque(maxlen=batch_size + 1)
  for _ in range(batch_size + 1):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(training_size):
    list_buffer = list(buffer)
    training_data[i] = list_buffer[:batch_size]
    labels[i] = list_buffer[1:]
    data_index = (data_index + 1) % len(data)
    for _ in range(batch_size):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
  return training_data, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

number_of_batch = 1000
batch_size = 10
training_data, labels = generate_batch(training_size=number_of_batch, batch_size = batch_size)
print('toaol training size:', len(training_data) * batch_size)
print('label size:', len(labels) * batch_size)
print('example training batch data:', [[reverse_dictionary[b] for b in bi] for bi in batch[:2]])
print('example labels:',[[reverse_dictionary[l] for l in li] for li in labels[:2]])

In [8]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0) 
class RNNNumpy:
     
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
    
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def calculate_total_loss(self, x, y):
        L = 0
        # For each sentence...
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            # We only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # Add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L
 
    def calculate_loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x,y)/N

In [9]:
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(training_data[0]) # input batch_size words
print(o.shape)
print(o)

(10, 15000)
[[  6.62109333e-05   6.67196519e-05   6.67217783e-05 ...,   6.67782746e-05
    6.66192580e-05   6.67231913e-05]
 [  6.64688245e-05   6.68617663e-05   6.64587208e-05 ...,   6.66221047e-05
    6.66575393e-05   6.65015954e-05]
 [  6.66438125e-05   6.69261478e-05   6.67110563e-05 ...,   6.69424599e-05
    6.61166672e-05   6.66485979e-05]
 ..., 
 [  6.69452754e-05   6.64347871e-05   6.66880558e-05 ...,   6.67230928e-05
    6.64866970e-05   6.68044492e-05]
 [  6.68432246e-05   6.70532657e-05   6.62952360e-05 ...,   6.64221627e-05
    6.61275837e-05   6.69638582e-05]
 [  6.67078676e-05   6.66390377e-05   6.67673229e-05 ...,   6.70055253e-05
    6.68277927e-05   6.65578990e-05]]


In [10]:
# Try predict, each word is batch[0] will predict a new word
predictions = model.predict(training_data[0])
print predictions.shape
print predictions

(10,)
[ 8384  3893 13076  6075  8203  3196  9363  8397 12661 11314]


In [11]:
# Try loss
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(training_data, labels)

Expected Loss for random predictions: 9.615805
Actual loss: 9.615732


In [12]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt

In [13]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is too large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
 
RNNNumpy.gradient_check = gradient_check
 
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


/.auto/home/ydong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [14]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNNNumpy.sgd_step = numpy_sdg_step

In [15]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [ ]:
# train a small model with 500 training examples
training_size = 500
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, training_data[:training_size], labels[:training_size], nepoch=10, evaluate_loss_after=1)

2016-07-02 17:28:30: Loss after num_examples_seen=0 epoch=0: 9.615685
2016-07-02 17:30:40: Loss after num_examples_seen=500 epoch=1: 9.606484
2016-07-02 17:32:45: Loss after num_examples_seen=1000 epoch=2: 8.597782
2016-07-02 17:34:59: Loss after num_examples_seen=1500 epoch=3: 7.280509
2016-07-02 17:37:07: Loss after num_examples_seen=2000 epoch=4: 6.920833
2016-07-02 17:39:22: Loss after num_examples_seen=2500 epoch=5: 6.749891
2016-07-02 17:41:39: Loss after num_examples_seen=3000 epoch=6: 6.634353
2016-07-02 17:43:46: Loss after num_examples_seen=3500 epoch=7: 6.565943
2016-07-02 17:45:59: Loss after num_examples_seen=4000 epoch=8: 6.502040
2016-07-02 17:48:09: Loss after num_examples_seen=4500 epoch=9: 6.440839


In [ ]:
# Generate next words given history words
def generate_sentence(model, start_word, sent_len):
    # We start the sentence with the start token
    new_sentence = [start_word]
    # Repeat until we get an the sent len we want
    while not len(new_sentence) == sent_len:
        next_word_probs, s = model.forward_propagation(new_sentence)
        sampled_word = np.argmax(next_word_probs[-1])
        new_sentence.append(sampled_word)
    sentence_str = [reverse_dictionary[x] for x in new_sentence]
    return sentence_str
num_sentences = 20
sent_min_length = 5
# Randomly got 10 words from vocab as start word
np.random.seed(10)
initial_words = np.random.random_integers(0, vocabulary_size, (num_sentences))
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    sent = generate_sentence(model, initial_words[i], sent_min_length)
    print " ".join(sent)

## Question 2
When you want to apply LSTM(Long­Short Term Memory) instead of vanilla RNN, what would you consider more in terms of cell and hidden states, and several gates?

** answer: **

LSTM network is designed to deal with vanishing gradients through a gating mechanism. Basically there will be a *input gate*, *output gate*, and *forget gate*. They are matrixes with value between 0 to 1. 

- the forget gate defines how much of the previous state you want to let through
- the input gate defines how much of the newly computed state for current input youwant to let through
- the output gate defines how much of the internal state you want to expose to the external network

Given the current input and prev state, a cell can calculate the current hidden state. All gates has the same dimentions with the size of hidden state

## Question 3
If you try to use the pretrained word2vec, what parts in your structure have to be changed, and what will be the expected effect?

** answer: **

If we want to use a pre-trained word embedding, we can change th one hot vocab to word embedding vocab. The current
*dictionary* will change to a map from word string to word embedding vector, the *reverse_dictionary* will be a map from word embedding vector to a string. Use a pre-trained word embedding is useful when the training data is small.

If we don't want to use a pre-trained one, we can also add a embedding layer between the one hot word vector and hidden layer. By update the embedding layer during training, we can also learn a word vectors, but it will be less general.

By using the pretrained word vector, we will expect faster updating speed and lower loss, because now we know the relationship between each words, the model can learn one word and generalize to the other similar words.

## Question 4
What could you expect when dropout is applied to feedforward? How would you implement it?


** answer: **

Dropout can be applied to avoid overfitting. Dropout is applied to feedforword in RNNs and the recurrent connections are kept untouched. 

To implement it, we need to randomly drop the output from hidden layer to output layer. In order to do that, for the output matrix V, during training, if we set the dropout rate to be 0.5, everytime we need to randomly select half the the rows in V and set them to be zero. So those hidden layer correspond to those rows will not affect the output layer. 

## Question 5

When it comes to prediction, could you consider beam search? What else could you suggest to dynamically generate sentences?

Yes, beam search is a heuristic search algorithm to find a sub optimal when it's impossible/hard to find a global optimal. For a k-beam search, we keep k candidates with largest scores in the beam, everytime we do a prediction for every candidate in the beam and choose the largest k candidates and replace the old ones in the beam. In this way we can explore more states then greedy search. Gready search is actully a beam search with beam size euqals 1.

Other way could be randomly select one or more next state based on their scores/probs. This will make the results more diverse.

### References
[1](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/) RECURRENT NEURAL NETWORKS TUTORIAL, PART 2  IMPLEMENTING A RNN WITH PYTHON, NUMPY AND THEANO

[2](https://www.tensorflow.org/versions/r0.9/tutorials/recurrent/index.html) Recurrent Neural Networks - Tensorflow

[3](http://sebastianruder.com/optimizing-gradient-descent/)An overview of gradient descent optimization algorithms